In [3]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd

In [6]:
ls data/data_raw

activity.csv           regions_hcps.csv       split.csv
hcps.csv               rtes.csv               submission_sample.csv
regions.csv            sales_train.csv


In [ ]:
ls

In [8]:
ac = pd.read_csv('data/data_raw/activity.csv')
hcps = pd.read_csv('data/data_raw/hcps.csv')
reg_hcps = pd.read_csv('data/data_raw/regions_hcps.csv')
sales = pd.read_csv('data/data_raw/sales_train.csv')

In [13]:
ac.hcp.value_counts()

32137    45
59771    44
79877    37
26277    37
55843    36
         ..
50037     1
49478     1
14274     1
48857     1
82349     1
Name: hcp, Length: 15819, dtype: int64

In [15]:
ac.specialty.value_counts()

General practicioner                          25349
Internal medicine / pneumology                14536
Internal medicine                             12533
Pediatrician                                   1129
Internal medicine and general practicioner      391
Name: specialty, dtype: int64

In [26]:
hcps.groupby('specialty').hcp.count()

specialty
General practicioner                          1861
Internal medicine                             1227
Internal medicine / pneumology                1766
Internal medicine and general practicioner      49
Pediatrician                                    36
Name: hcp, dtype: int64

In [30]:
hcps.region.value_counts()

region_52     75
region_200    75
region_150    71
region_131    69
region_88     60
              ..
region_100     6
region_141     6
region_125     6
region_104     5
region_56      3
Name: region, Length: 201, dtype: int64

In [31]:
hcps

,hcp,tier,region,specialty
0,83,2,region_59,Internal medicine
1,112,1,region_103,Internal medicine
2,307,2,region_33,Internal medicine / pneumology
3,387,2,region_97,Internal medicine / pneumology
4,426,1,region_118,General practicioner
...,...,...,...,...
4934,93927,2,region_67,Pediatrician
4935,93928,2,region_25,Pediatrician
4936,93929,2,region_126,Pediatrician
4937,93930,1,region_17,Internal medicine / pneumology


In [29]:
sales.brand.unique()

array(['brand_1', 'brand_2', 'brand_3', 'brand_12_market',
       'brand_3_market'], dtype=object)

In [ ]:
sales.groupby('month')